In [15]:
from core import process_new_files, daily_summary_from_hist   

df_raw = process_new_files( # te pedirá el capital inicial en ventana emergente
    merge_fragments=False, 
    verbose=True)

# QA : Quality Assurance
from core.qa import check_integrity
#check_integrity(df_raw)

print(df_raw.head().T)

INFO: 📦 Total trades cargados: 72
INFO: 💾 Datos guardados en PostgreSQL (trades).
INFO: ✅ Procesados 62 trades en total.


                                                       0  \
trade_id                                               1   
entry_time                           2025-06-16 09:32:54   
exit_time                            2025-06-16 09:49:16   
asset                                          MESM5@CME   
entry_price                                       6022.0   
exit_price                                        6033.5   
position_size                                          3   
PnL                                               -172.5   
PnL_net                                           -172.5   
commission                                             0   
account                                   APEX-298755-19   
exchange                                             CME   
direction                                           Sell   
order_id_entry                                   1053256   
order_id_exit                                    1071398   
source_file       ATAS_statistics_160620

In [17]:
from core.merger import merge_split_trades
from core.io_utils import save_to_csv, save_to_postgres
from core.summary import daily_summary_from_hist


df = merge_split_trades(df_raw)
print(df.head().T)

# 2️⃣  Guardas el DataFrame en un CSV y en Postgres
# output/trades_hist.csv
save_to_csv(df)
save_to_postgres(df)

INFO: 💾 Datos guardados en PostgreSQL (trades).


                                                       0  \
entry_time                           2025-06-09 12:50:30   
exit_time                            2025-06-09 12:51:00   
asset                                          MNQM5@CME   
direction                                           Sell   
entry_price                                     21860.25   
exit_price                                      21867.75   
position_size                                          4   
PnL                                                -60.0   
PnL_net                                            -60.0   
commission                                             0   
order_id_entry                                   1232061   
order_id_exit                                    1235556   
account                                   APEX-298755-19   
exchange                                             CME   
source_file       ATAS_statistics_08062025_30062025.xlsx   
components                              

In [3]:
# Métricas globales rápidas
df_summary = daily_summary_from_hist(df)

total_fragmented = df["components"].apply(len).gt(1).sum() if "components" in df.columns else 0

print("\nTrades totales :", len(df))
print(f"Número de trades fragmentados: {total_fragmented}")
print("Win-rate       :", (df['PnL_net'] > 0).mean())
print("Profit factor  :", df.loc[df['PnL_net'] > 0, 'PnL_net'].sum() /
                          df.loc[df['PnL_net'] < 0, 'PnL_net'].abs().sum())
print("Drawdown máx % :", 1 - df['equity'].div(df['equity'].cummax()).min())
print("Total PnL_net   :", df['PnL_net'].sum())
print("Última equity   :", df['equity'].iloc[-1])

df_summary


Trades totales : 53
Número de trades fragmentados: 5
Win-rate       : 0.5283018867924528
Profit factor  : 1.37227478403949
Drawdown máx % : 0.2973262032085562
Total PnL_net   : 226.25
Última equity   : 645.25


,day,total_trades,net_pnl,win_rate,fragmented_trades,fragmented_ids,profit_factor
0,2025-06-09,11,-81.00,0.545455,3,"[2, 4, 5]",NaN
1,2025-06-10,9,204.50,0.555556,1,[13],NaN
2,2025-06-11,12,100.75,0.666667,0,[],NaN
3,2025-06-12,11,-144.50,0.272727,1,[36],NaN
4,2025-06-13,10,146.50,0.600000,0,[],NaN
5,TOTAL,53,226.25,0.528302,5,"[2, 4, 5, 13, 36]",1.372275


In [4]:
# 2️⃣  Calcula P y R desde df con la nueva función
from core.kelly_analysis import (
    extraer_parametros_desde_df,
    simular_trayectorias_monte_carlo,
    calcular_expected_shortfall,
    resumen_simulacion,
    test_validacion_binomial
)

P, R, meta = extraer_parametros_desde_df(df)
print(f"P = {P:.4f}  \nR = {R:.2f}  \nmeta = {meta}")

# 3️⃣  Decide la fracción de Kelly y lanza Monte Carlo
from core.kelly_simulator_v4 import calcular_kelly

kelly_puro = calcular_kelly(P, R)
f = 1.0                    # 100 % de Kelly
n_trades = 100
paths = 10_000
cap0 = df['equity'].iloc[-1]

eq = simular_trayectorias_monte_carlo(P, R, kelly_puro * f,
                                      cap0=cap0, n=n_trades, n_paths=paths)

print("\nResumen:", resumen_simulacion(eq))
es, p5 = calcular_expected_shortfall(eq)
print(f"\nCVaR 5 %: {es:.2f} (p5 = {p5:.2f})")

# 4️⃣  Test binomial (opcional, si quieres validar el win-rate)
wins = meta['wins']
losses = meta['losses']
p_val = test_validacion_binomial(wins, losses, p_teorico=P)
print(f"\nP-valor binomial: {p_val:.4f}")


P = 0.5283  
R = 1.23  
meta = {'wins': 28, 'losses': 25, 'p_mean': np.float64(0.5283018867924528), 'r_mean': np.float64(1.2252453428924017)}



Resumen: {'p05': 189.00561011145757, 'p25': 670.2534030572355, 'p50': 2376.8586765487275, 'p75': 6142.23756201655, 'p95': 29890.42052748302, 'mean': 7498.633586266826, 'std': 19963.608242240316, 'min': 4.238202445416743, 'max': 707852.6403924761}

CVaR 5 %: 102.78 (p5 = 189.01)

P-valor binomial: 1.0000


# Interpretación científica de las métricas de simulación con estrategia Kelly

Este documento explica el significado de las principales variables utilizadas en el análisis cuantitativo de una estrategia de trading basada en el criterio de Kelly. Está dirigido a investigadores, estudiantes y analistas interesados en el modelado probabilístico del riesgo y rendimiento de sistemas de trading.

---

### Variables fundamentales

#### P (win-rate): `P = 0.5283`

Es la **probabilidad empírica de acierto**: el cociente entre el número de trades con beneficio (`wins = 28`) y el total de trades (`28 + 25 = 53`). Este valor se interpreta como una estimación puntual de la probabilidad de que un trade resulte positivo.

#### R (Win/Loss ratio): `R = 1.23`

Representa la **relación entre el beneficio medio de los trades ganadores y la pérdida media de los perdedores**. Es una medida crucial del "payoff" relativo y sirve para determinar si una estrategia puede ser rentable incluso con tasas de acierto modestas.

#### Meta (información auxiliar):

```python
{
  'wins': 28,
  'losses': 25,
  'p_mean': 0.5283,
  'r_mean': 1.2252
}
```

Valores que resumen el rendimiento histórico, utilizados como entrada para simulaciones posteriores.

---

### Resultados de la simulación Monte Carlo (10,000 trayectorias)

Se simuló la evolución del capital inicial (645 USD) durante 100 operaciones usando la fracción de Kelly teórica.

#### Percentiles de capital final:

* **p05**: 189.01 → el 5% de los escenarios termina con menos de \$189.
* **p25**: 670.25 → un cuartil está por debajo de este valor.
* **p50**: 2376.86 → mediana. Capital más probable tras 100 trades.
* **p75**: 6142.24 → el 25% superior supera este valor.
* **p95**: 29890.42 → resultados excepcionalmente buenos.

#### Media y dispersión:

* **mean**: 7498.63 → valor esperado de capital final (sujeto a asimetría).
* **std**: 19963.61 → desviación estándar de los capitales finales.
* **min**: 4.24 → peor resultado observado (cuasi-ruina).
* **max**: 707,852.64 → mejor escenario extremo.

---

### CVaR (Conditional Value at Risk) al 5%

* **p5**: 189.01 → umbral que delimita el peor 5% de escenarios.
* **CVaR 5 %**: 102.78 → media del capital final entre los peores casos. Refleja la **pérdida esperada condicional a que ocurra un evento adverso severo**.

---

### Validación empírica

* **Test binomial** con hipótesis nula: `p_real = P`
* Resultado: `p_val = 1.0000` → No hay evidencia para rechazar que la tasa de acierto empírica sea igual a la teórica.

---

### Conclusión

La estrategia presenta un perfil de retorno asimétrico con alta dispersión y posibilidad de crecimiento exponencial. Sin embargo, también exhibe escenarios de ruina parcial, como evidenciado por el CVaR. La interpretación de estos resultados debe hacerse considerando el contexto temporal, la estabilidad de la estrategia y su sensibilidad a cambios estructurales del mercado.


In [1]:
import core.global_state as gs
gs.df.columns


AttributeError: 'NoneType' object has no attribute 'columns'

In [2]:
import importlib
import core.kelly_simulator_v4_1 as ks
importlib.reload(ks)

ks.mostrar_interfaz()

⚠️  Falling back to defaults: No DataFrame válido en global_state
⚠️  Falling back to defaults: No DataFrame válido en global_state


In [6]:
from core.utils import load_df_for_analysis
import core.global_state as state

state.df = load_df_for_analysis()


In [ ]:
from core.kelly.simulator import mostrar_interfaz
mostrar_interfaz()


In [ ]:

# carga y ejecuta el simulador
import importlib
import core.kelly_simulator_v4 as ks
importlib.reload(ks)
ks.mostrar_interfaz()

Output()

Output()

Output()

In [ ]:

import core.kelly_simulator_v5 as ks
importlib.reload(ks)

# 3️⃣  Lanzar la interfaz
ks.mostrar_interfaz()
